In [1]:
%pip install tesserocr pillow

  Using cached tesserocr-2.7.1.tar.gz (71 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [36 lines of output]
      pkg-config failed to find tesseract/leptonica libraries: Package 'tesseract' was not found
      
      Failed to extract tesseract version from executable: [WinError 2] The system cannot find the file specified
      Supporting tesseract v3.04.00
      Tesseract major version 3
      Traceback (most recent call last):
        File "<string>", line 239, in get_build_args
        File "<string>", line 120, in package_config
      Exception: Package 'tesseract' was not found
      
      
      During handling of the above exception, another exception occurred:
      
      Traceback (most recent call last):
        File "e:\7th sem\GO_GO_Nihongo\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "e:\7th sem\GO_GO_Nihongo\venv\Lib\site-packages\pip\_vend

In [2]:
from PIL import Image
import tesserocr

# Load an image
image = Image.open("./image.png")

# Initialize Tesseract with Japanese language
with tesserocr.PyTessBaseAPI(lang='jpn') as api:
    api.SetImage(image)
    
    # Get bounding boxes along with recognized text
    boxes = api.GetComponentImages(tesserocr.RIL.TEXTLINE, True)
    for i, (im, box, _, _) in enumerate(boxes):
        api.SetRectangle(box['x'], box['y'], box['w'], box['h'])
        text = api.GetUTF8Text().strip()
        print(f"Text: {text}, Coordinates: {box}")


ModuleNotFoundError: No module named 'tesserocr'